In [ ]:

def meta_grid(df, code_of_interest, thresh_list, train_test):
    # With Sentiment ####
    ## all words
    review_vs_whole_ref_bag(df,code_of_interest, True, "whole", thresh_list, train_test, "allwords", True)
    review_sentence_vs_whole_ref_bag(df,code_of_interest, True, "whole", thresh_list, train_test, "allwords", True)
    review_sentence_vs_ref_sentence(df,code_of_interest, True, "sentence", thresh_list, train_test, "allwords", True)
    # stop words
    review_vs_whole_ref_bag(df,code_of_interest, True, "whole", thresh_list, train_test, "stopwords", True)
    review_sentence_vs_whole_ref_bag(df,code_of_interest, True, "whole", thresh_list, train_test, "stopwords", True)
    review_sentence_vs_ref_sentence(df,code_of_interest, True, "sentence", thresh_list, train_test, "stopwords", True)

    # Without Sentiment ####
    ## allwords
    review_vs_whole_ref_bag(df,code_of_interest, False, "whole", thresh_list, train_test, "allwords")
    review_sentence_vs_whole_ref_bag(df,code_of_interest, False, "whole", thresh_list, train_test, "allwords", True)
    review_sentence_vs_ref_sentence(df,code_of_interest, False, "sentence", thresh_list, train_test, "allwords", True)
    ##stopwords
    review_vs_whole_ref_bag(df,code_of_interest, False, "whole", thresh_list, train_test, "stopwords", True)
    review_sentence_vs_whole_ref_bag(df,code_of_interest, False, "whole", thresh_list, train_test, "stopwords", True)
    review_sentence_vs_ref_sentence(df,code_of_interest, False, "sentence", thresh_list, train_test, "stopwords", True)



